<a href="https://colab.research.google.com/github/mtdiedrich/NFL-Fantasy-Projection/blob/main/OPOY_Upside.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Get the HTML table from URL

from bs4 import BeautifulSoup
import urllib.request

url = 'https://www.vegasinsider.com/nfl/odds/player-of-the-year/'
response = urllib.request.urlopen(url)
html = response.read()
soup = BeautifulSoup(html)

# Get only the table
opoy_table = soup.find('tbody', id='see-all-offensive-player-of-the-year')

opoy_table

<tbody class="drawer active" data-group="" id="see-all-offensive-player-of-the-year"> <tr class="divided" data-name="saquon barkley"> <td class="game-team"> <img data-role="imagable" data-src="https://assets.actionnetwork.com/200x200/00b49214b0094e6a8b4718b2b9a50033.png" height="32" src="https://bctn-vi.s3.amazonaws.com/img/user.svg?v=c357130" width="32"/> <span>Saquon Barkley</span> </td> <td class="game-odds"> <a class="" href="https://switchboard.vegasinsider.com/v2/deeplink?deeplinkId%5B0%5D=ML1316925813&amp;context=vi-qs-odds" rel="nofollow" target="_blank"> <span class="data-value"> +650 </span> <span class="button-img button-badge"> <svg height="20" style="background-color:#AE996A;" viewbox="-7.5 -7.5 35 35" width="20" xmlns="http://www.w3.org/2000/svg"> <path d="M17 17H3V3h5V1H3a2 2 0 0 0-2 2v14a2 2 0 0 0 2 2h14a2 2 0 0 0 2-2v-5h-2z" fill="#fff" stroke="#fff"></path> <text fill="#fff" font-size="16px" font-weight="bold" x="10" y="10">+</text> </svg> </span> </a> </td> <td class

In [ ]:
#@title Parse OPOY odds from table

tr_tags = opoy_table.find_all('tr')

player_odds = {}

for tr_tag in tr_tags:
  td_tags = tr_tag.find_all('td')

  for td_tag in td_tags:
    tag_value = td_tag.text
    tag_value = tag_value.replace('+', '')
    tag_value = tag_value.strip()

    if tag_value:
      try:
        odds = int(tag_value)
        player_odds[player_name].append(odds)
      except ValueError:
        player_name = tag_value
        player_odds[player_name] = []

player_odds

{'Saquon Barkley': [650, 750, 650, 650],
 "Ja'Marr Chase": [800, 850, 900, 800],
 'Jahmyr Gibbs': [1000, 1000, 1200, 1300],
 'Derrick Henry': [1400, 1200, 1500, 1400],
 'Bijan Robinson': [1400, 1400, 1400, 1400],
 'Justin Jefferson': [1600, 1600, 1600, 1200],
 'Puka Nacua': [2000, 1800, 2500, 2000],
 'CeeDee Lamb': [2000, 2000, 2200, 2000],
 'Christian McCaffrey': [1600, 2000, 1600, 1500],
 'Nico Collins': [1800, 2000, 1600, 1600],
 'Amon-Ra St. Brown': [3000, 3000, 3500, 2500],
 'Brian Thomas': [3500, 3500, 3500, 3500],
 'Malik Nabers': [3500, 3500, 3500, 2500],
 'Tyreek Hill': [3000, 3500, 4000, 4000],
 'Jayden Daniels': [5000, 4000, 4000, 5000],
 'Jonathan Taylor': [4000, 4000, 3500, 4200],
 'Marvin Harrison': [6600, 5000, 6000],
 'Josh Allen': [5000, 5000, 4000, 6000],
 'A.J. Brown': [5000, 5000, 5000, 4000],
 'Bucky Irving': [5000, 5000, 5000, 6000],
 'Joe Burrow': [5000, 5000, 4000, 6000],
 'Lamar Jackson': [5000, 5000, 4000, 6000],
 "De'Von Achane": [5000, 5000, 5000, 3500],
 'D

In [ ]:
#@title Calculate measures of central tendency for each player's set of odds

from statistics import median, geometric_mean
import numpy as np

player_odds_data = {}

for key, value in player_odds.items():

  if value:
    player_odds_data[key] = {
        'Median': median(value),
        'Geometric Mean': geometric_mean(value)
    }
  else:
    player_odds_data[key] = {
        'Median': np.nan,
        'Geometric Mean': np.nan
    }

player_odds_data

{'Saquon Barkley': {'Median': 650.0, 'Geometric Mean': 673.674848441361},
 "Ja'Marr Chase": {'Median': 825.0, 'Geometric Mean': 836.4892272955084},
 'Jahmyr Gibbs': {'Median': 1100.0, 'Geometric Mean': 1117.5865065755215},
 'Derrick Henry': {'Median': 1400.0, 'Geometric Mean': 1370.5097960612409},
 'Bijan Robinson': {'Median': 1400.0, 'Geometric Mean': 1399.9999999999998},
 'Justin Jefferson': {'Median': 1600.0, 'Geometric Mean': 1488.9677745633596},
 'Puka Nacua': {'Median': 2000.0, 'Geometric Mean': 2059.7671439071178},
 'CeeDee Lamb': {'Median': 2000.0, 'Geometric Mean': 2048.2273781688905},
 'Christian McCaffrey': {'Median': 1600.0,
  'Geometric Mean': 1664.7165801151264},
 'Nico Collins': {'Median': 1700.0, 'Geometric Mean': 1742.3508698771448},
 'Amon-Ra St. Brown': {'Median': 3000.0, 'Geometric Mean': 2978.9460677644734},
 'Brian Thomas': {'Median': 3500.0, 'Geometric Mean': 3499.9999999999995},
 'Malik Nabers': {'Median': 3500.0, 'Geometric Mean': 3217.629503287215},
 'Tyreek H

In [ ]:
#@title Create the implied probability dataframe

import pandas as pd

df = pd.DataFrame(player_odds_data).T

# the implied probability that the player wins OPOY
df['Implied Probability'] = 1 / (df['Geometric Mean']/100 + 1)

# adjust for vig using uniform scaling
df['P(X=OPOY)'] = df['Implied Probability'] / df['Implied Probability'].sum()

df = df.sort_values('P(X=OPOY)', ascending=False)

df.head()

,Median,Geometric Mean,Implied Probability,P(X=OPOY)
Saquon Barkley,650.0,673.674848,0.129253,0.065439
Ja'Marr Chase,825.0,836.489227,0.106782,0.054062
Jahmyr Gibbs,1100.0,1117.586507,0.082130,0.041581
Derrick Henry,1400.0,1370.509796,0.068004,0.034429
Bijan Robinson,1400.0,1400.000000,0.066667,0.033753


In [ ]:
!pip install shin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.1/235.1 kB 3.3 MB/s eta 0:00:00


In [ ]:
#@title Adjust for vig using the Shin method

shin_df = df[['Geometric Mean', 'Implied Probability']].copy()

# assign an arbitary small probability for players without odds
shin_df['Implied Probability'] = shin_df['Implied Probability'].fillna(.001)

# the shin method requires decimal odds
shin_df['Implied Decimal Odds'] = 1 / shin_df['Implied Probability']

from shin import calculate_implied_probabilities

shin_df['Shin Implied Probability'] = calculate_implied_probabilities(
    shin_df['Implied Decimal Odds'])


shin_df['Shin Decimal Odds'] = 1 / shin_df['Shin Implied Probability']

shin_df = shin_df.sort_values('Shin Implied Probability', ascending=False)

pd.set_option('display.max_rows', 100)

shin_df.head(100)

,Geometric Mean,Implied Probability,Implied Decimal Odds,Shin Implied Probability,Shin Decimal Odds
Saquon Barkley,673.674848,0.129253,7.736748,0.088436,11.307596
Ja'Marr Chase,836.489227,0.106782,9.364892,0.072465,13.799789
Jahmyr Gibbs,1117.586507,0.082130,12.175865,0.054953,18.197376
Derrick Henry,1370.509796,0.068004,14.705098,0.044927,22.258520
Bijan Robinson,1400.000000,0.066667,15.000000,0.043978,22.738555
Justin Jefferson,1488.967775,0.062934,15.889678,0.041331,24.195065
Christian McCaffrey,1664.716580,0.056666,17.647166,0.036888,27.109226
Nico Collins,1742.350870,0.054278,18.423509,0.035196,28.412248
CeeDee Lamb,2048.227378,0.046550,21.482274,0.029726,33.641104
Puka Nacua,2059.767144,0.046301,21.597671,0.029550,33.841364


In [ ]:
#@title Get an ADP dataframe
adp_url = 'https://fantasyfootballcalculator.com/api/v1/adp/ppr?teams=12&year=2025'

adp_response = urllib.request.urlopen(adp_url)
adp_response_value = adp_response.read()

import json

adp_json = json.loads(adp_response_value.decode('utf-8'))
adp_df = pd.DataFrame(adp_json['players'])

adp_df.head()

,player_id,name,position,team,adp,adp_formatted,times_drafted,high,low,stdev,bye
0,5177,Ja'Marr Chase,WR,CIN,1.5,1.02,1474,1,5,0.8,10
1,5670,Bijan Robinson,RB,ATL,2.1,1.02,985,1,5,0.8,5
2,2860,Saquon Barkley,RB,PHI,2.4,1.02,542,1,6,0.9,9
3,4876,Justin Jefferson,WR,MIN,4.4,1.04,832,2,9,1.2,6
4,4869,CeeDee Lamb,WR,DAL,4.9,1.05,1193,1,9,1.5,10


In [ ]:
#@title Apply basic adjustments

# Match names to formats from odds data
adp_df['name'] = adp_df['name'].apply(lambda x: x.replace(' Jr.', ''))
adp_df['name'] = adp_df['name'].apply(lambda x: x.replace(' Sr.', ''))
adp_df['name'] = adp_df['name'].apply(lambda x: x.replace(' III', ''))
adp_df['name'] = adp_df['name'].apply(lambda x: x.replace(' II', ''))
adp_df['name'] = adp_df['name'].apply(lambda x: x.replace('Cameron Ward', 'Cam Ward'))
adp_df['name'] = adp_df['name'].apply(lambda x: x.replace('Marquise Brown', 'Hollywood Brown'))

adp_df.index = adp_df['name']
adp_df = adp_df.drop('name', axis=1)

# Drop non-offensive positions
adp_df = adp_df[~adp_df['position'].isin(['DEF', 'PK'])]

adp_df.head()

,player_id,position,team,adp,adp_formatted,times_drafted,high,low,stdev,bye
name,,,,,,,,,,
Ja'Marr Chase,5177,WR,CIN,1.5,1.02,1474,1,5,0.8,10
Bijan Robinson,5670,RB,ATL,2.1,1.02,985,1,5,0.8,5
Saquon Barkley,2860,RB,PHI,2.4,1.02,542,1,6,0.9,9
Justin Jefferson,4876,WR,MIN,4.4,1.04,832,2,9,1.2,6
CeeDee Lamb,4869,WR,DAL,4.9,1.05,1193,1,9,1.5,10


In [ ]:
#@title Calculate Probability-ADP Rank Delta

joined_df = shin_df.join(adp_df, how='outer')[['Shin Implied Probability',
                                               'position', 'adp']]

joined_df['Shin Rank'] = joined_df['Shin Implied Probability'].rank(
    ascending=False)

joined_df['ADP Rank'] = joined_df['adp'].rank()

joined_df = joined_df.dropna()

joined_df['delta'] = joined_df['ADP Rank'] - joined_df['Shin Rank']

joined_df = joined_df.sort_values('delta', ascending=False)

pd.set_option('display.max_rows', None)

# higher delta = more under-valued

joined_df

,Shin Implied Probability,position,adp,Shin Rank,ADP Rank,delta
Cam Ward,0.001708,QB,164.7,86.0,185.0,99.0
Andrei Iosivas,0.001453,WR,166.1,94.0,187.0,93.0
C.J. Stroud,0.003873,QB,128.8,46.5,130.0,83.5
Michael Penix,0.002081,QB,152.2,78.5,160.0,81.5
Sam Darnold,0.001382,QB,161.4,99.5,180.0,80.5
Joe Mixon,0.003783,RB,128.4,48.0,128.0,80.0
Jordan Love,0.003955,QB,121.3,45.0,122.0,77.0
Justin Herbert,0.005532,QB,108.4,37.0,112.0,75.0
Bryce Young,0.001686,QB,152.8,88.0,162.5,74.5
Aaron Rodgers,0.000972,QB,168.4,119.0,193.0,74.0


In [ ]:
#@title Calculate value over last starter (VOLS)

LAST_STARTER = {
    'QB': 20,
    'RB': 30,
    'WR': 30,
    'TE': 20
}

def calculate_last_starter(data, position):
  pos_df = data[data['position']==position].copy()
  pos_df['Pos Rank'] = pos_df['Shin Implied Probability'].rank(ascending=False)
  rep_pos = pos_df[pos_df['Pos Rank'] >= LAST_STARTER[position]]
  if len(rep_pos) > 0:
    rep_shin = max(rep_pos['Shin Implied Probability'].values)
  else:
    rep_shin = 0
  pos_df['Last Starter Diff'] = pos_df['Shin Implied Probability'] - rep_shin
  return pos_df


last_starter_dfs = [calculate_last_starter(joined_df, key) for key
                    in LAST_STARTER.keys()]
ls_df = pd.concat(last_starter_dfs)
ls_df = ls_df.sort_values('Last Starter Diff', ascending=False)
ls_df.head()

,Shin Implied Probability,position,adp,Shin Rank,ADP Rank,delta,Pos Rank,Last Starter Diff
Saquon Barkley,0.088436,RB,2.4,1.0,3.0,2.0,1.0,0.087511
Ja'Marr Chase,0.072465,WR,1.5,2.0,1.0,-1.0,1.0,0.069979
Jahmyr Gibbs,0.054953,RB,5.1,3.0,6.0,3.0,2.0,0.054028
Derrick Henry,0.044927,RB,10.8,4.0,12.0,8.0,3.0,0.044001
Bijan Robinson,0.043978,RB,2.1,5.0,2.0,-3.0,4.0,0.043053


In [ ]:
#@title Round-by-round projection breakdown

pick_number = 7

adp_df = ls_df.sort_values('adp').copy()

# Calculates pick numbers in a snake draft
my_picks = [int(pick_number * (-1)**i + 20 * (i%2 + i) + ((-1)**(i + 1) + 1) / 2) for i in range(10)]

def get_vonr_df(data, next_data, pos):
  # Calculates a player's value over the best player at the same position that
  # is expected to be avaiable at the user's next pick, according to ADP
  # VONR = value over next round
  pick_df = data.copy()
  next_pick_df = next_data.copy()
  next_top_pos = next_pick_df[next_pick_df['position']==pos]
  next_pos_max = max(next_top_pos['Last Starter Diff'].values)
  pos_pick_df = pick_df[pick_df['position']==pos].copy()
  pos_pick_df['VONR'] = pos_pick_df['Last Starter Diff'] - next_pos_max
  return pos_pick_df

# for each pick, print out the top 25 players that are expected to be available
for idx in range(len(my_picks)):
  pick = my_picks[idx]
  try:
    next_pick = my_picks[idx+1]
  except IndexError:
    break

  pick_df = adp_df[adp_df['ADP Rank'] >= pick]
  next_pick_df = adp_df[adp_df['ADP Rank'] >= next_pick]

  pos_frames = []
  for pos_key in ['QB', 'RB', 'WR', 'TE']:
    try:
      pos_frames.append(get_vonr_df(pick_df, next_pick_df, pos_key))
    except ValueError:
      continue

  vonr_df = pd.concat(pos_frames).sort_values('VONR', ascending=False)

  print('===== PICK:', pick, '=====')
  print(vonr_df.head(25))
  print()
  print()
  print()

===== PICK: 7 =====
                     Shin Implied Probability position   adp  Shin Rank  ADP Rank  delta  Pos Rank  Last Starter Diff      VONR
Derrick Henry                        0.044927       RB  10.8        4.0      12.0    8.0       3.0           0.044001  0.036676
Christian McCaffrey                  0.036888       RB   7.7        7.0       8.0    1.0       5.0           0.035963  0.028637
Nico Collins                         0.035196       WR   9.8        8.0       9.0    1.0       3.0           0.032710  0.025781
Puka Nacua                           0.029550       WR  10.0       10.0      10.0    0.0       5.0           0.027064  0.020134
Amon-Ra St. Brown                    0.019798       WR  10.6       11.0      11.0    0.0       6.0           0.017312  0.010383
Malik Nabers                         0.018157       WR   7.5       12.0       7.0   -5.0       7.0           0.015671  0.008742
Brian Thomas                         0.016500       WR  14.9       13.0      15.0   